<a href="https://colab.research.google.com/github/fursew05/Dacon/blob/main/%EC%9E%AC%EB%AC%B4%EC%A7%80%ED%91%9C%ED%86%B5%ED%95%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### 재무 데이터 + 비재무 데이터 + 거시경제 데이터 만들기

In [ ]:
# 회계년도 컬럼 정수형으로 변경 및 추가여부 컬럼삭제
df = pd.read_csv("data\파생위한재무종합.csv",index_col=0)
df['회계년도'] = df['회계년도'].str.split('/').str[0]
df['회계년도'] = df['회계년도'].astype('int64')
df.drop(columns=['연결_추가여부', '개별_추가여부', 'kos_gaap_추가여부','비상장_연결_추가여부', '비상장_개별_추가여부', '비상장_gaap_추가여부'],inplace=True)

In [ ]:
# 데이터 조인하기
df1 = pd.read_csv('data\이자비용처리 전 데이터.csv',index_col=0)
join_data = df1.merge(df,how='left',on=['회사명','거래소코드','회계년도'])

In [ ]:
# 이자보상배율이 999인 행 삭제
join_data = join_data[join_data['이자보상배율(이자비용)'] != 999999999]

In [ ]:
# 각 행마다 어떤 회계기준 재무데이터를 가져왔는지에 대한 컬럼 추가
join_data['GAAP'] = join_data['kos_gaap_추가여부'] + join_data['out_gaap_1_추가여부'] + join_data['out_gaap_2_추가여부'] + \
join_data['out_gaap_3_추가여부'] + join_data['out_gaap_4_추가여부']
join_data.rename(columns={'연결_추가여부' : 'IFRS_CONN','개별_추가여부' : 'IFRS'},inplace=True)

In [ ]:
# 업력이 음수인 데이터 제거
flag = join_data['업력'] >= 0
join_data = join_data.loc[flag]

# 추가여부 컬럼 삭제
drop_cols = ['kos_gaap_추가여부',
 'out_gaap_1_추가여부',
 'out_gaap_2_추가여부',
 'out_gaap_3_추가여부',
 'out_gaap_4_추가여부']
join_data.drop(columns=drop_cols,inplace=True)

# 회계년도가 2012년 이후인 데이터만 추출
eda_data = join_data[join_data['회계년도']>2011]

In [ ]:
# object 타입으로 변경
object_cols = [
'IFRS_CONN',
 'IFRS',
'대표이사변경여부',
 '수도권','부실여부','GAAP']

for col in object_cols:
    eda_data[col] = eda_data[col].astype('object')

C:\Users\fursew\AppData\Local\Temp\ipykernel_20104\659736143.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eda_data[col] = eda_data[col].astype('object')
C:\Users\fursew\AppData\Local\Temp\ipykernel_20104\659736143.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eda_data[col] = eda_data[col].astype('object')
C:\Users\fursew\AppData\Local\Temp\ipykernel_20104\659736143.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [ ]:
# 회계년도의 연속여부 컬럼 추가
import pandas as pd
def year_check(x: pd.Series) -> int:
    return int(len(x) != (x.max() - x.min() + 1))

eda_data['회계년도연속여부'] = eda_data.groupby(['회사명', '거래소코드'])['회계년도'].transform(
    lambda x: year_check(x)
)
eda_data['회계년도연속여부'] = eda_data['회계년도연속여부'].astype('object')

C:\Users\fursew\AppData\Local\Temp\ipykernel_20104\1509938036.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eda_data['회계년도연속여부'] = eda_data.groupby(['회사명', '거래소코드'])['회계년도'].transform(
C:\Users\fursew\AppData\Local\Temp\ipykernel_20104\1509938036.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eda_data['회계년도연속여부'] = eda_data['회계년도연속여부'].astype('object')


In [ ]:
# 무한대 값 변환
eda_data.replace([np.inf, -np.inf], np.nan, inplace=True)

C:\Users\fursew\AppData\Local\Temp\ipykernel_20104\3690913638.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  eda_data.replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\fursew\AppData\Local\Temp\ipykernel_20104\3690913638.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eda_data.replace([np.inf, -np.inf], np.nan, inplace=True)


In [ ]:
# 이자보상배율 컬럼 제거
eda_data.drop(columns=[ '이자보상배율(이자비용)',
 '이자보상배율(순금융비용)',
'이자보상배율(영업이익)'
],inplace=True)

C:\Users\fursew\AppData\Local\Temp\ipykernel_20104\1827388257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eda_data.drop(columns=[ '이자보상배율(이자비용)',


In [ ]:
eda_data.to_csv('data/eda/EDA데이터_거시포함.csv',index=False)

### Dickinson 국면 분할

In [ ]:
### 국면으로 나누기
op_col = '영업현금흐름(천원)'
inv_col = '투자활동으로 인한 현금흐름(*)(천원)'
fin_col = '재무활동으로 인한 현금흐름(*)(천원)'

# 국면 분류 함수 정의
def classify_dickinson(row):
    op = row[op_col]
    inv = row[inv_col]
    fin = row[fin_col]

    if op < 0 and inv < 0 and fin > 0:
        return '도입기'
    elif op > 0 and inv < 0 and fin > 0:
        return '성장기'
    elif op > 0 and inv < 0 and fin < 0:
        return '성숙기'
    elif op < 0 and inv > 0:
        return '쇠퇴기'
    else:
        return '기타'

# 데이터프레임에 생애주기 컬럼 추가
eda_data['생애주기'] = eda_data.apply(classify_dickinson, axis=1)

# 원하는 순서 정의
stage_order = ['도입기', '성장기', '성숙기', '쇠퇴기', '기타']

# value_counts 후 원하는 순서로 정렬
counts = eda_data['생애주기'].value_counts().loc[stage_order]

# 출력
print(counts)
print("\n총 기업 수:", counts.sum())

filtered_lifecycle = eda_data[eda_data['생애주기'] != '기타'].copy()

df_도입 = filtered_lifecycle[filtered_lifecycle['생애주기']=='도입기'].drop('생애주기',axis=1)
df_성장 = filtered_lifecycle[filtered_lifecycle['생애주기']=='성장기'].drop('생애주기',axis=1)
df_성숙 = filtered_lifecycle[filtered_lifecycle['생애주기']=='성숙기'].drop('생애주기',axis=1)
df_쇠퇴 = filtered_lifecycle[filtered_lifecycle['생애주기']=='쇠퇴기'].drop('생애주기',axis=1)

C:\Users\fursew\AppData\Local\Temp\ipykernel_20104\1459615397.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eda_data['생애주기'] = eda_data.apply(classify_dickinson, axis=1)


생애주기
도입기    42314
성장기    53975
성숙기    68220
쇠퇴기    23977
기타     41454
Name: count, dtype: int64

총 기업 수: 229940


In [ ]:
df_도입.to_csv('data/eda/eda도입.csv',index=False)
df_성장.to_csv('data/eda/eda성장.csv',index=False)
df_성숙.to_csv('data/eda/eda성숙.csv',index=False)
df_쇠퇴.to_csv('data/eda/eda쇠퇴.csv',index=False)